*Disclaimer*: This notebook borrows from different sources including the IBMs tutorial on [data embeddings](https://learn.qiskit.org/course/machine-learning/data-encoding).

# Data Embedding and Encoding

## Basis Encoding

Qiskit allows to create a valid basis encoding circuit by using the `initialize` function. In case we want to create a circuit that represents a dataset $X = \{5, 7\}$ that maps to the quantum state $|X\rangle = \frac{1}{\sqrt{2}}(|101\rangle + |111\rangle)$ we can use the following:

In [ ]:
import math
from qiskit import QuantumCircuit

desired_state = [
    0,
    0,
    0,
    0,
    0,
    1 / math.sqrt(2),
    0,
    1 / math.sqrt(2)
]
qc = QuantumCircuit(3)
qc.initialize(desired_state, range(3))
qc.decompose().decompose().decompose().decompose().decompose().draw("mpl")
# qc.draw("mpl")

## Ensuring normalization of quantum states

Please remember that we always need to ensure that $|a|^2 = 1$. Please take care that you also need to account for complex numbers.

### Exercise

Consider a dataset $X$ that contains the following data points: $\{2, 3, 5, 5, -1\}$ and encode it using *basis encoding*.

In [ ]:
# Enter your code here

## Amplitude Encoding

As an example we encode the dataset $X = \{ x_1 = (1.5, 0), x_2=(-2, 3) \}$ with amplitude encoding.

While concatenating the 2 features of our 2 data points we get the following:

$$\alpha = \frac{1}{\sqrt(15.25)}(1.5, 0, -2, 3)$$

and our resulting 2-qubit state is therefore

$$|X\rangle = \frac{1}{\sqrt{15.25}}(1.5|00\rangle - 2|10\rangle + 3|11\rangle)$$

As we already did in basis encoding, we can again use the initialize function to let qiskit create our desired state.

In [ ]:
from typing import List
import numpy as np


def a_norm(values: List[float]) -> float:
    """Calculates the constant `a_norm`"""
    return 1 / math.sqrt(np.sum(np.abs(values)**2))

In [ ]:
values = [1.5, 0, -2, 3]
normalization = a_norm(values)

In [ ]:
desired_state

In [ ]:

desired_state = [normalization * value for value in values]
qc = QuantumCircuit(2)
qc.initialize(desired_state, range(2))
qc.decompose().decompose().decompose().decompose().decompose().draw("mpl")
# qc.draw("mpl")

## Angle Encoding

We now want to encode the data point $x = (0, \frac{\pi}{4}, \frac{\pi}{2})$ using angle encoding.
Remembering that $U(x_j^i) = R_Y(2x_j^i)$ we can set up the following circuit:

In [ ]:
values = [0, np.pi / 4, np.pi / 2]

In [ ]:
qc = QuantumCircuit(3)
for qubit, value in enumerate(values):
    qc.ry(2 * value, qubit)
qc.draw("mpl")

## Arbitrary Encoding

In arbitrary encoding we use parameterised gates to encode up to $N$ features of one data point. To properly encode all features, we need at least $N$ parameterised gates but those can be distributed over $n$ qubits, where $n \leq N$. As seen last week, qiskit provides several Ansätze with parameterised gates that we can also use for arbitrary encoding.

For example the [`EfficientSU2`](https://qiskit.org/documentation/stubs/qiskit.circuit.library.EfficientSU2.html) circuit can encode up to $12$ features on $3$ qubits.

In [ ]:
from qiskit.circuit.library import EfficientSU2
circuit = EfficientSU2(num_qubits=3, reps=1, insert_barriers=True)
circuit.decompose().draw("mpl")

We can now bin a given data point to the just created circuit.

In [ ]:
x = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2]
encode = circuit.bind_parameters(x)
encode.decompose().draw("mpl")

There are also Ansätze that support less parameters for binding than parameterised gates contained in the circuit. This may be the case when e.g. combinations of different features are encoded in addition to support higher order functions. In the following, we use the [`ZZFeatureMap`](https://qiskit.org/documentation/stubs/qiskit.circuit.library.ZZFeatureMap.html) on $3$ qubits. Although it contains $6$ parameterised gates, it only supports $3$ parameters.

In [ ]:
from qiskit.circuit.library import ZZFeatureMap
circuit = ZZFeatureMap(3, reps=1, insert_barriers=True)
circuit.decompose().draw("mpl")

In [ ]:
x = [0.1, 0.2, 0.3]
encode = circuit.bind_parameters(x)
encode.decompose().draw("mpl")

## Creating a QRAM

A quantum random access memory (QRAM) is a data structure allowing to store a series of $n$-qubit states that are addressible via an index. Let us imagine we want to store eight different states. Each state is generated through an operator $A_i$. Our QRAM encodes the different states by generating the following state:

$$ \frac{1}{8} (|0\rangle A_0 |0\rangle^{\otimes n} + |1\rangle A_1 |0\rangle^{\otimes n} + \dots + |7\rangle A_7 |0\rangle^{\otimes n})$$

In this case, we need to store eight elements in memory. Therefore, we need three qubits each for address and memory registers.

The potential of QRAM lies in the fact that we can work with all data simultaneously as they are stored in superposition. This for example allows complex Grover searches or obtaining better performance in different QML algorithms.

In the following, we want to store the values $X = \{ 3, 6\}$. As we only want to store $2$ values, $1$ address register is sufficient, while we may consider $3$ qubits for storing our values.

In [ ]:
from qiskit import QuantumRegister

address = QuantumRegister(1, "a")
memory = QuantumRegister(3, "m")
qram = QuantumCircuit(address, memory)
qram.h(address)
qram.cx(address, memory[0])
qram.cx(address, memory[2])
qram.x(address)
qram.cx(address, memory[0])
qram.cx(address, memory[1])
qram.x(address)
qram.draw("mpl")

### Exercise

The operator $A_i$ is the gate $R_Y(\theta)$ and you want to store a list of $8$ elements of angles $\theta_i$.

In [ ]:
# Add your code here